In [18]:
from pynsee.localdata import *
from pynsee.geodata import *
from pynsee.metadata import *
from shapely import wkt

import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

# get a list all data available : datasets and variables
metadata = get_local_metadata()

# geographic metadata
nivgeo = get_nivgeo_list()

# get geographic area list
area = get_area_list()

In [2]:
# get departement map
dep = get_geodata('ADMINEXPRESS-COG-CARTO.LATEST:departement')
dep = dep[["insee_dep", "geometry"]]
# remove overseas departments
dep = dep[~dep['insee_dep'].isin(['971', '972', '973', '974', '976'])]
dep = dep.rename(columns={'insee_dep': 'CODEGEO'})

Previously saved data used
/home/jovyan/.cache/pynsee/pynsee/a28d68dc88fcd55fe62626ea25a1760a.csv
Set update=True to get the most up-to-date data


In [3]:
data = get_local_data(dataset_version = 'GEO2020FLORES2017',
                     variables =  'NA17',
                     nivgeo = 'DEP')

Getting departements: 100%|██████████| 18/18 [00:33<00:00,  1.87s/it]


By default, the query is on all departements


Getting data:   7%|▋         | 7/102 [00:01<00:16,  5.64it/s]


API query number limit reached - function might be slowed down


Getting data: 100%|██████████| 102/102 [03:33<00:00,  2.10s/it]


!!! This underlying API is still at an early development stage,
future changes are likely !!!


In [4]:
df = data[data["UNIT"] == "NBSALET"].reset_index(drop=True)
df = df[df["NA17"] != "ENS"].reset_index(drop=True)

In [5]:
# compute share of activity in total employment by department
df['employment_share'] = 100 * df["OBS_VALUE"] / df['OBS_VALUE'].groupby(df['CODEGEO']).transform('sum')
df['employment_share']

0        1.281165
1        2.515339
2        0.000000
3        0.180870
4        0.030145
          ...    
1712     2.297605
1713     0.937722
1714     8.525423
1715    36.085478
1716     4.018007
Name: empl_share, Length: 1717, dtype: float64

In [14]:
# merge values and geographic limits
datamap = dep.merge(df, how = 'left', on = 'CODEGEO')
datamap = gpd.GeoDataFrame(datamap).set_crs("EPSG:4326")

In [29]:
activity = get_activity_list("NAF2")
activity.head(90)
activity.to_csv("activity.csv")

In [25]:
activity = get_activity_list("A38")
activity

,A10,A21,A38,TITLE_A38_FR,TITLE_A38_EN
0,AZ,A,AZ,"Agriculture, sylviculture et pêche","Agriculture, forestry and fishing"
1,BE,B,BZ,Industries extractives,Mining and quarrying
2,BE,C,CA,"Fabrication de denrées alimentaires, de boisso...","Manufacture of food products, beverages and to..."
3,BE,C,CB,"Fabrication de textiles, industries de l'habil...","Manufacture of textiles, wearing apparel, leat..."
4,BE,C,CC,"Travail du bois, industries du papier et impri...",Manufacture of wood and paper products; printi...
5,BE,C,CD,Cokéfaction et raffinage,Manufacture of coke and refined petroleum prod...
6,BE,C,CE,Industrie chimique,Manufacture of chemicals and chemical products
7,BE,C,CF,Industrie pharmaceutique,Manufacture of basic pharmaceutical products a...
8,BE,C,CG,Fabrication de produits en caoutchouc et en pl...,"Manufacture of rubber and plastics products, a..."
9,BE,C,CH,Métallurgie et fabrication de produits métalli...,Manufacture of basic metals and fabricated met...


In [16]:
def get_center(x):
    return x.centroid.wkt
datamap["center"] = datamap['geometry'].apply(lambda x: get_center(x))


In [17]:
datamap

,CODEGEO,geometry,NIVGEO,UNIT,UNIT_label,NA17,OBS_VALUE,NA17_label,DATASET_VERSION,DATASET_NAME,DATA_DATE,GEO_DATE,CODEGEO_label,empl_share,center
0,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,AZ,1577.0,"Agriculture, sylviculture et pêche",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,0.845141,POINT (5.348728661460733 46.09976825936063)
1,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C1,4964.0,"Fabrication de denrées alimentaires, de boisso...",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,2.660293,POINT (5.348728661460733 46.09976825936063)
2,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C2,0.0,Cokéfaction et raffinage,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,0.000000,POINT (5.348728661460733 46.09976825936063)
3,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C3,7195.0,"Fabrication d'équipements électriques, électro...",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,3.855924,POINT (5.348728661460733 46.09976825936063)
4,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C4,4242.0,Fabrication de matériels de transport,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,2.273361,POINT (5.348728661460733 46.09976825936063)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,KZ,7234.0,Activités financières et d'assurance,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,1.865601,POINT (2.130831812063975 49.08273891355998)
1645,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,LZ,3334.0,Activités immobilières,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,0.859817,POINT (2.130831812063975 49.08273891355998)
1646,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,MN,48690.0,Activités scientifiques et techniques ; servic...,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,12.556833,POINT (2.130831812063975 49.08273891355998)
1647,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,OQ,124402.0,"Administration publique, enseignement, santé h...",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,32.082464,POINT (2.130831812063975 49.08273891355998)
